# GSL PERIODOGRAMS - TESS LC BATCH FILE PROCESSING

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

This notebook takes the GTO objects table and calculates the Lomb-Scargle periodogram for each of the available TESS light curves, storing the results in `FITS` files with a primary header and two additional header data units, one with the light curve itself and another with the periodogram.

Additionally, the GTO objects table and file is updated with new columns storing the results for each object.

Error control is implemented to prevent exceptions and timeouts to interrupt the processing. In case any of these errors happen, it is so recorded in the results table. Timeout value is dinamically updated, as the calculation for each record is completed.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $3 h$, so we will set the limits of the periodogram frequencies for periods between $5 min$ and $10 h$. In days, this corresponds to a range of $P\in[0.003472, 0.416667] d$ or, equivalently, frequencies in the range $f\in[288.0, 2.4] d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [2]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

import warnings

#from scipy import stats

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [3]:
# Configuration:
# Files and folders:
GTO_FILE = "../data/CARM_VIS_objects_with_PG.csv"
IN_LC_FOLDER = "../data/CARMENES_GTO_TESS_lc/lightcurves/"
OUT_IMG_FOLDER = "../data/CARM_VIS_TESS_PGs/figures/"
OUT_PROCESSED_FOLDER = "../data/CARM_VIS_TESS_PGs/"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Periodogram constants:
# Periodogram constants:
FBEG = 2.4 # d^{-1}, corresponds to a period P=10 hours
FEND = 288 # d^{-1}, corresponds to a period P=5 min
NUM_POINTS = 285601 # 285600 for TESS LC time-series
PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
#FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
FREQ = np.linspace(start=FBEG, stop=FEND, num=NUM_POINTS)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
LS = False # Default
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# PULSATION RANGE OF INTEREST (FOR THE PLOTS):
F_LOW = 8 # P=0.1250 d (3 h)
F_HIGH = 72 # P=0.0139 d (20 min)

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


In [4]:
def tess_lc_load(filename: str):
    '''Load the TESS LC file and returns a lightcurve object with just the three relevant columns
    Note: it seems to be necessary to drop 'nan' values for GLS to work properly.'''
    lc_lk = lk.read(filename).remove_nans()
    return lc_lk

In [5]:
def lc_infer_sampling(lc_lk: lk.LightCurve):
    '''Infer sampling period from light curve'''
    time_diffs = lc_lk['time'][1:] - lc_lk['time'][:-1]
    return np.median(time_diffs)

## Data processing

### GTO data loading

In [6]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_T0_PG_TESS,WF_e_T0_PG_TESS,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000997,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.689208,0.027575,999.999756,2.151008e-06,1.0,1.0,NaN,94.758838,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23548+38...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1353.926624,0.036481,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN


In [7]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

In [8]:
gto.loc[72, ['Karmn', 'has_lc', 'lc_file']]

Karmn                                             J19025+754
has_lc                                                  True
lc_file    ../data/CARMENES_GTO_TESS_lc/lightcurves/J1902...
Name: 72, dtype: object

Generate the proper auxiliary columns (with the basic periodograms results).

In [9]:
# Additional columns:
if 'n_TESS' in gto.columns:
    pass
else:
    gto['n_TESS'] = None # Number of points in TESS LC
    gto['Ps_TESS'] = None # Sampling period (d)
    gto['fs_TESS'] = None # Sampling frequency (d^(-1))
    gto['wmean_TESS'] = None # Mean of TESS LC
    gto['wrms_TESS'] = None # RMS of TESS LC
    gto['info_PG_TESS'] = None # Information text about the PG
    gto['maxP_PG_TESS'] = None # Max power value in the PG
    gto['maxSNR_PG_TESS'] = None # Max power value in the PG
    gto['rms_PG_TESS'] = None # RMS value in the PG residuals
    gto['f_PG_TESS'] = None # Best frequency in the PG (d^(-1))
    gto['e_f_PG_TESS'] = None # Error of the best frequency in the PG (d^(-1))
    gto['Pd_PG_TESS'] = None # Best period in the PG (d)
    gto['e_Pd_PG_TESS'] = None # Error of the best period in the PG (d)
    gto['Ph_PG_TESS'] = None # Best period in the PG (hours)
    gto['e_Ph_PG_TESS'] = None # Error of the best period in the PG (hours)
    gto['Pm_PG_TESS'] = None # Best period in the PG (minutes)
    gto['e_Pm_PG_TESS'] = None # Error of the best period in the PG (minutes)
    gto['A_PG_TESS'] = None # Amplitude of the best frequency
    gto['e_A_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['ph_PG_TESS'] = None # Amplitude of the best frequency
    gto['e_ph_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['T0_PG_TESS'] = None # Reference epoch of the best frequency
    gto['e_T0_PG_TESS'] = None # Error of the epoch of the best frequency
    gto['offset_PG_TESS'] = None # Offset of the best frequency
    gto['e_offset_PG_TESS'] = None # Error of the offset of the best frequency
    gto['FAP_PG_TESS'] = None # False alarm probability
    gto['valid_PG_TESS'] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['error_PG_TESS'] = None # The error raised during processing. Empty if processing was successful.
    gto['elapsed_time_PG_TESS'] = None # The time elapsed in calculation
    gto['fits_file_TESS'] = None # The name of the processed file.
    gto['fig_file_TESS'] = None # The name of the figure file.


In [10]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Batch processing of all TESS files

In [11]:
n = len(gto)
n

390

#### Batch processing

In [12]:
# Prevent warnings to produce too much output
warnings.filterwarnings('ignore')

# Batch processing:
lapse_list = []
median_lapse = None
#for i in range(0, 3): # FOR TESTING
#for i in [123, 167]: # To repeat the calculation for specific objects.
#for i in range(0, 100):
#for i in range(100, 200):
#for i in range(200, 300):
for i in range(300, len(gto)):
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'Karmn'] # KARMENES name
    commn = gto.loc[i, 'Name'] # Common name
    tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    print("Record: %d, started at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star (TIC: %s)..." %(karmn, tic_id))
    try:
        # LOAD LC FILE:
        lc_file = gto.loc[i, 'lc_file']
        print("filename: %s" %lc_file)
        lc_lk = tess_lc_load(lc_file)
        
        # GENERATE PERIODOGRAM:
        gls = Gls((lc_lk['time'].value, lc_lk['flux'].value, lc_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)
        
        # GENERATE THE FITS FILE:
        # Prepare the fits primary HDU (only header):
        primary_header = fits.Header()
        primary_header['OBJECT'] = (karmn, "KARMENES target name")
        primary_header['NAME'] = (commn, "Object common name")
        primary_header['TIC'] = (tic_id, "Object TESS identifier")
        primary_header['RA_J2000'] = ("00:05:10.89", "Object right ascension (J2000)")
        primary_header['DE_J2000'] = ("+45:47:11.6", "Object declination (J2000)")
        primary_header['SPTYPE'] = ("M1.0 V", "Spectral type")
        primary_header['TEFF_K'] = (3773, "Effective temperature in Kelvin")
        primary_header['LOGG'] = (5.07, "Logarithm of surface gravity")
        primary_header['FEH'] = (-0.04, "Metallicity")
        primary_header['L_LSUN'] = (0.0436229, "Luminosity in Solar luminosities")
        primary_header['R_RSUN'] = (0.48881, "Radius in Solar radii")
        primary_header['M_MSUN'] = (0.4918, "Mass in Solar masses")
        primary_header['D_PC'] = (11.50352803, "Distance in parsec")
        hdu_primary = fits.PrimaryHDU(header=primary_header)

        # Prepare the LC HDU:
        hdu_lc = fits.table_to_hdu(QTable(lc_lk.to_table()))
        hdu_lc.name = "TESS_LC"
        psample = lc_infer_sampling(lc_lk).value
        fsample = 1.0 / psample
        fnyq = 2.0 * fsample
        freq_units = u.d ** (-1)
        hdu_lc.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_lc.header['PUNIT'] = u.d.to_string(format='fits')
        hdu_lc.header['FUNIT'] = freq_units.to_string(format='fits')
        hdu_lc.header['RVPOINTS'] = (gls.N, "Number of points in the TESS curve")
        hdu_lc.header['AVGFLUX'] = (gls._Y, "Average flux of TESS curve")
        hdu_lc.header['RMSFLUX'] = (np.sqrt(gls._YY), "Flux RMS of TESS curve")
        hdu_lc.header['PSAMPLE'] = (psample, "Inferred cadence in TESS curve")
        hdu_lc.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in TESS curve")
        hdu_lc.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
                
        # Prepare the PG HDU:
        hdu_pg = fits.table_to_hdu(
            QTable(data=[gls.freq, gls.power], names=['freq', 'power'], 
                   units=[1.0 / u.d, (u.electron / u.s) ** 2]))
        hdu_pg.name = "GLS_PG"
        fpoints = len(gls.f)
        fres = (gls.fend - gls.fbeg) / (fpoints - 1)
        hdu_pg.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_pg.header['FUNIT'] = (freq_units.to_string(format='fits'), "Unit for frequencies")
        hdu_pg.header['PUNIT'] = (u.d.to_string(format='fits'), "Unit for periods")
        hdu_pg.header['PK_FREQ'] = (gls.best['f'], "Frequency of the peak in periodogram")
        hdu_pg.header['PK_POW'] = (gls.pmax, "Power of the peak in periodogram")
        hdu_pg.header['PK_SNR'] = (gls.best['amp'] / gls.rms, "SNR of the peak in periodogram")
        hdu_pg.header['PK_FAP'] = (gls.FAP(Pn=None), "FAP of the peak in periodogram")
        hdu_pg.header['RES_RMS'] = (gls.rms, "RMS of residuals in periodogram")
        hdu_pg.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in TESS curve")
        hdu_pg.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
        hdu_pg.header['FPOINTS'] = (fpoints, "Number of points in periodogram")
        hdu_pg.header['FBEG'] = (gls.fbeg, "Start frequency in periodogram")
        hdu_pg.header['FEND'] = (gls.fend, "End frequency in periodogram")
        hdu_pg.header['FRES'] = (fres, "Frequency resolution in periodogram")
        hdu_pg.header['F'] = (gls.best['f'], "Peak best estimate: frequency")
        hdu_pg.header['E_F'] = (gls.best['e_f'], "Peak best estimate: frequency error")
        hdu_pg.header['P'] = (gls.best['P'], "Peak best estimate: period")
        hdu_pg.header['E_P'] = (gls.best['e_P'], "Peak best estimate: period error")
        hdu_pg.header['A'] = (gls.best['amp'], "Peak best estimate: amplitude")
        hdu_pg.header['E_A'] = (gls.best['e_amp'], "Peak best estimate: amplitude error")
        hdu_pg.header['PH'] = (gls.best['ph'], "Peak best estimate: phase")
        hdu_pg.header['E_PH'] = (gls.best['e_ph'], "Peak best estimate: phase error")
        hdu_pg.header['T0'] = (gls.best['T0'], "Peak best estimate: frequency")
        hdu_pg.header['E_T0'] = (gls.best['e_T0'], "Peak best estimate: frequency error")
        hdu_pg.header['OFF'] = (gls.best['offset'], "Peak best estimate: offset")
        hdu_pg.header['E_OFF'] = (gls.best['e_offset'], "Peak best estimate: offset error")
        hdu_pg.header['OFAC'] = (gls.ofac, "Setup: oversampling factor")
        hdu_pg.header['HIFAC'] = (gls.ofac, "Setup: maximum frequency factor")
        hdu_pg.header['NORM'] = (gls.norm, "Setup: normalization type")
        hdu_pg.header['LS'] = (gls.ls, "Setup: conventional Lomb-Scargle calculation")
        hdu_pg.header['FAST'] = (gls.fast, "Setup: fast evaluation, recursive trigonometric")
        
        # Create and save the fits file:
        fits_filename = OUT_PROCESSED_FOLDER + karmn + "_TESS_PG.fits"
        hdul = fits.HDUList([hdu_primary, hdu_lc, hdu_pg])
        hdul.writeto(fits_filename, overwrite=True)
        
        # Delete all the HDUs:
        del hdul
        del hdu_primary
        del hdu_lc
        del hdu_pg

        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'n_TESS'] = gls.N
        gto.loc[i, 'Ps_TESS'] = psample
        gto.loc[i, 'fs_TESS'] = fsample
        gto.loc[i, 'wmean_TESS'] = gls._Y
        gto.loc[i, 'wrms_TESS'] = np.sqrt(gls._YY)
        gto.loc[i, 'info_PG_TESS'] = gls.info(stdout=False)
        gto.loc[i, 'maxP_PG_TESS'] = gls.power.max()
        gto.loc[i, 'maxSNR_PG_TESS'] = gls.best['amp'] / gls.rms
        gto.loc[i, 'rms_PG_TESS'] = gls.rms
        gto.loc[i, 'f_PG_TESS'] = gls.best['f']
        gto.loc[i, 'e_f_PG_TESS'] = gls.best['e_f']
        gto.loc[i, 'Pd_PG_TESS'] = gls.best['P']
        gto.loc[i, 'e_Pd_PG_TESS'] = gls.best['e_P']
        gto.loc[i, 'Ph_PG_TESS'] = 24.0 * gls.best['P']
        gto.loc[i, 'e_Ph_PG_TESS'] = 24.0 * gls.best['e_P']
        gto.loc[i, 'Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'e_Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'A_PG_TESS'] = gls.best['amp']
        gto.loc[i, 'e_A_PG_TESS'] = gls.best['e_amp']
        gto.loc[i, 'ph_PG_TESS'] = gls.best['ph']
        gto.loc[i, 'e_ph_PG_TESS'] = gls.best['e_ph']
        gto.loc[i, 'T0_PG_TESS'] = gls.best['T0']
        gto.loc[i, 'e_T0_PG_TESS'] = gls.best['e_T0']
        gto.loc[i, 'offset_PG_TESS'] = gls.best['offset']
        gto.loc[i, 'e_offset_PG_TESS'] = gls.best['e_offset']
        gto.loc[i, 'FAP_PG_TESS'] = gls.FAP(Pn=None)

        # GENERATE THE FIGURE:
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        # Add the reference lines for predicted pulsations:
        fig.axes[0].axvline(F_LOW, color="darkgray", linestyle="--")
        fig.axes[0].axvline(F_HIGH, color="darkgray", linestyle="--")
        figure_title = "TESS LC: %s (TIC %s), P=%.4f [min], FAP=%.4f%%" \
            %(karmn, tic_id, gto.loc[i, 'Pm_PG_TESS'], 100.0 * gto.loc[i, 'FAP_PG_TESS'])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + karmn + "_TESS_PG.jpg"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'valid_PG_TESS'] = 1
        gto.loc[i, 'error_PG_TESS'] = ""
        gto.loc[i, 'fits_file_TESS'] = fits_filename
        gto.loc[i, 'fig_file_TESS'] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'elapsed_time_PG_TESS'] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")

        # Clear memory (delete the 'gls' object'):
        try:
            del gls
        except:
            pass

    except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "--- Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'valid_PG_TESS'] = 0
        gto.loc[i, 'error_PG_TESS'] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'elapsed_time_PG_TESS'] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "--- Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'error_PG_TESS'] = gto.loc[i, 'error_PG_TESS'] + "/" + str(e2)

        # Clear memory (delete the 'gls' object'):
        try:
            del gls
        except:
            pass


Record: 300, started at 20/08/2022, 06:42:23
Previous median lapse time: None
Processing J05421+124 star (TIC: 247463344)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J05421+124/J05421+124_TESS_TIC247463344_S06.fits
Elapsed time: 114.55 seconds
... SUCCESS.
Record: 301, started at 20/08/2022, 06:44:17
Previous median lapse time: 114.55 seconds
Processing J05415+534 star (TIC: 311064351)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J05415+534/J05415+534_TESS_TIC311064351_S19.fits
Elapsed time: 10.85 seconds
... SUCCESS.
Record: 302, started at 20/08/2022, 06:44:28
Previous median lapse time: 62.70 seconds
Processing J05394+406 star (TIC: 239097694)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J05394+406/J05394+406_TESS_TIC239097694_S19.fits
Elapsed time: 125.79 seconds
... SUCCESS.
Record: 303, started at 20/08/2022, 06:46:34
Previous median lapse time: 114.55 seconds
Processing J05366+112 star (TIC: 436248798)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcu

Elapsed time: 189.46 seconds
... SUCCESS.
Record: 329, started at 20/08/2022, 07:35:31
Previous median lapse time: 125.79 seconds
Processing J04219+213 star (TIC: 17307715)...
filename: nan
--- Some ERROR happened with record #329, J04219+213 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 330, started at 20/08/2022, 07:35:31
Previous median lapse time: 125.46 seconds
Processing J04198+425 star (TIC: 347994537)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J04198+425/J04198+425_TESS_TIC347994537_S19.fits
Elapsed time: 183.04 seconds
... SUCCESS.
Record: 331, started at 20/08/2022, 07:38:34
Previous median lapse time: 125.79 seconds
Processing J04173+088 star (TIC: 283866910)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J04173+088/J04173+088_TESS_TIC283866910_S05.fits
Elapsed time: 196.49 seconds
... SUCCESS.
Record: 332,

Elapsed time: 223.69 seconds
... SUCCESS.
Record: 358, started at 20/08/2022, 08:39:20
Previous median lapse time: 155.89 seconds
Processing J02222+478 star (TIC: 409527090)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J02222+478/J02222+478_TESS_TIC409527090_S18.fits
Elapsed time: 225.74 seconds
... SUCCESS.
Record: 359, started at 20/08/2022, 08:43:06
Previous median lapse time: 156.14 seconds
Processing J02164+135 star (TIC: 408587159)...
filename: nan
--- Some ERROR happened with record #359, J02164+135 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 360, started at 20/08/2022, 08:43:06
Previous median lapse time: 155.89 seconds
Processing J02123+035 star (TIC: 270415707)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J02123+035/J02123+035_TESS_TIC270415707_S04.fits
Elapsed time: 216.19 seconds
... SUCCESS.
Record: 361

Elapsed time: 231.98 seconds
... SUCCESS.
Record: 368, started at 20/08/2022, 08:58:22
Previous median lapse time: 155.89 seconds
Processing J01433+043 star (TIC: 369534824)...
filename: nan
--- Some ERROR happened with record #368, J01433+043 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 369, started at 20/08/2022, 08:58:22
Previous median lapse time: 155.64 seconds
Processing J01352-072 star (TIC: 29853348)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01352-072/J01352-072_TESS_TIC29853348_S03.fits


Elapsed time: 223.72 seconds
... SUCCESS.
Record: 370, started at 20/08/2022, 09:02:06
Previous median lapse time: 155.89 seconds
Processing J01339-176 star (TIC: 54003038)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01339-176/J01339-176_TESS_TIC54003038_S03.fits


Elapsed time: 217.42 seconds
... SUCCESS.
Record: 371, started at 20/08/2022, 09:05:44
Previous median lapse time: 156.14 seconds
Processing J01125-169 star (TIC: 610210976)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01125-169/J01125-169_TESS_TIC439403362_S03.fits
Elapsed time: 226.81 seconds
... SUCCESS.
Record: 372, started at 20/08/2022, 09:09:31
Previous median lapse time: 156.34 seconds
Processing J01066+192 star (TIC: 243185500)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01066+192/J01066+192_TESS_TIC243185500_S17.fits


Elapsed time: 245.39 seconds
... SUCCESS.
Record: 373, started at 20/08/2022, 09:13:36
Previous median lapse time: 156.54 seconds
Processing J01056+284 star (TIC: 15615674)...
filename: nan
--- Some ERROR happened with record #373, J01056+284 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 374, started at 20/08/2022, 09:13:36
Previous median lapse time: 156.34 seconds
Processing J01048-181 star (TIC: 268633473)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01048-181/J01048-181_TESS_TIC268633473_S03.fits
Elapsed time: 226.02 seconds
... SUCCESS.
Record: 375, started at 20/08/2022, 09:17:22
Previous median lapse time: 156.54 seconds
Processing J01033+623 star (TIC: 52183206)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J01033+623/J01033+623_TESS_TIC52183206_S18.fits
Elapsed time: 268.36 seconds
... SUCCESS.
Record: 376, s

Elapsed time: 269.93 seconds
... SUCCESS.
Record: 383, started at 20/08/2022, 09:52:24
Previous median lapse time: 170.79 seconds
Processing J00286-066 star (TIC: 40082634)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J00286-066/J00286-066_TESS_TIC40082634_S03.fits
Elapsed time: 250.10 seconds
... SUCCESS.
Record: 384, started at 20/08/2022, 09:56:34
Previous median lapse time: 172.55 seconds
Processing J00184+440 star (TIC: 440109718)...
filename: nan
--- Some ERROR happened with record #384, J00184+440 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 385, started at 20/08/2022, 09:56:34
Previous median lapse time: 170.79 seconds
Processing J00183+440 star (TIC: 440109725)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J00183+440/J00183+440_TESS_TIC440109725_S17.fits
Elapsed time: 266.65 seconds
... SUCCESS.
Record: 386, 

In [13]:
gto.loc[gto['lc_file'] == '<ambiguous>', ['Karmn']]

,Karmn


## Check the calculations that took longer to complete

We now check the calcultions that took longer, so as to try to repeat them (as it could be due to a problem with the computer itself - for example, the program stopped when the computer went into "sleep" mode).

In [14]:
gto[['Karmn', 'elapsed_time_PG_TESS']].sort_values(by='elapsed_time_PG_TESS', ascending=False) \
    .head(10)

,Karmn,elapsed_time_PG_TESS
298,J06000+027,409.811453
296,J06024+498,405.184140
297,J06011+595,386.011730
288,J06548+332,368.194157
292,J06318+414,367.667636
289,J06421+035,365.088064
295,J06103+821,363.638667
284,J07033+346,362.037632
294,J06105-218,350.003743
283,J07044+682,346.088966


In [16]:
gto[['Karmn', 'elapsed_time_PG_TESS', 'valid_PG_TESS']].sort_values(by='elapsed_time_PG_TESS', ascending=False) \
    .tail(10)

,Karmn,elapsed_time_PG_TESS,valid_PG_TESS
95,J18051-030,0.000971,0.0
45,J20556-140N,0.000969,0.0
68,J19098+176,0.000968,0.0
4,J23492+024,0.000967,0.0
9,J23340+001,0.000966,0.0
349,J02519+224,0.000966,0.0
356,J02358+202,0.000965,0.0
43,J20567-104,0.000964,0.0
387,J00162+198E,0.000963,0.0
106,J17303+055,0.000000,0.0


## Review a given file

In [17]:
object_name = "J23548+385"

In [18]:
read_hdul = fits.open(OUT_PROCESSED_FOLDER + object_name + "_TESS_PG.fits",
                      mode='update', memmap=False)
read_hdul

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x000001A2856EA3A0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001A2853A5190>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001A2FF40CEB0>]

In [19]:
read_hdul.info()

Filename: ../data/CARM_VIS_TESS_PGs/J23548+385_TESS_PG.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      17   ()      
  1  TESS_LC       1 BinTableHDU    216   12878R x 24C   [2D, E, E, E, J, D, D, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  GLS_PG        1 BinTableHDU     45   285601R x 2C   [D, D]   


### Retrieve basic information

In [20]:
read_hdul['PRIMARY'].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
OBJECT  = 'J23548+385'         / KARMENES target name                           
NAME    = 'RX J2354.8+3831'    / Object common name                             
TIC     = '291689873'          / Object TESS identifier                         
RA_J2000= '00:05:10.89'        / Object right ascension (J2000)                 
DE_J2000= '+45:47:11.6'        / Object declination (J2000)                     
SPTYPE  = 'M1.0 V  '           / Spectral type                                  
TEFF_K  =                 3773 / Effective temperature in Kelvin                
LOGG    =                 5.07 / Logarithm of surface gravity                   
FEH     =                -0.

### Retrieve RV curve information

In [21]:
read_hdul['TESS_LC']

In [22]:
read_hdul['TESS_LC'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  132 / length of dimension 1                          
NAXIS2  =                12878 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   24 / number of table fields                         
TIMESYS = 'TDB     '           / Default time scale                             
JDREF   =                  0.0 / Time columns are jd = jd1 + jd2                
TREFPOS = 'TOPOCENTER'         / Time reference position                        
TTYPE1  = 'time    '                                                            
TFORM1  = '2D      '        

In [23]:
read_hdul['TESS_LC'].data[:10]

FITS_rec([([ 2.45876500e+06, -3.10855424e-01], 8036.715 , 14.602657, 0.0047049 , 386805, 743.37405765, 1030.75688962, 7419.3154, 13.216773, 1896.0168, 6.645701 , 8036.715 , 14.602657, 0, nan, nan, nan, nan, 743.37405765, 0.00165376, 1030.75688962, 0.00156959, 0.04306674, 0.08566172),
          ([ 2.45876500e+06, -3.09466527e-01], 8045.7295, 14.630568, 0.00470491, 386806, 743.37714375, 1030.75655315, 7433.0312, 13.242034, 1900.8579, 6.6685295, 8045.7295, 14.630568, 0, nan, nan, nan, nan, 743.37714375, 0.00165215, 1030.75655315, 0.0015669 , 0.04319848, 0.08758464),
          ([ 2.45876500e+06, -3.08077632e-01], 8045.2686, 14.622687, 0.00470492, 386807, 743.38027732, 1030.75631546, 7430.282 , 13.234902, 1898.9921, 6.6640596, 8045.2686, 14.622687, 0, nan, nan, nan, nan, 743.38027732, 0.00165761, 1030.75631546, 0.00156569, 0.04801591, 0.08800385),
          ([ 2.45876500e+06, -3.06688735e-01], 8035.3765, 14.61121 , 0.00470493, 386808, 743.37142282, 1030.75155629, 7416.531 , 13.224514, 1900.

In [24]:
read_rv_table = QTable.read(read_hdul['TESS_LC'])
read_rv_table

time [2],flux,flux_err,timecorr,cadenceno,centroid_col,centroid_row,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
d,,,d,,pix,pix,,,,,,,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
float64,float32,float32,float32,int32,float64,float64,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
2458765.0 .. -0.3108554236009695,8.0367148e+03,1.4602657e+01,4.7049029e-03,386805,743.37406,1030.75689,7.4193154e+03,1.3216773e+01,1.8960168e+03,6.6457009e+00,8.0367148e+03,1.4602657e+01,0,nan,nan,nan,nan,743.37406,1.6537585e-03,1030.75689,1.5695895e-03,4.3066736e-02,8.5661724e-02
2458765.0 .. -0.30946652739862657,8.0457295e+03,1.4630568e+01,4.7049108e-03,386806,743.37714,1030.75655,7.4330312e+03,1.3242034e+01,1.9008579e+03,6.6685295e+00,8.0457295e+03,1.4630568e+01,0,nan,nan,nan,nan,743.37714,1.6521512e-03,1030.75655,1.5668995e-03,4.3198477e-02,8.7584637e-02
2458765.0 .. -0.3080776316621723,8.0452686e+03,1.4622687e+01,4.7049182e-03,386807,743.38028,1030.75632,7.4302822e+03,1.3234902e+01,1.8989921e+03,6.6640596e+00,8.0452686e+03,1.4622687e+01,0,nan,nan,nan,nan,743.38028,1.6576147e-03,1030.75632,1.5656911e-03,4.8015915e-02,8.8003851e-02
2458765.0 .. -0.3066887354593746,8.0353765e+03,1.4611210e+01,4.7049262e-03,386808,743.37142,1030.75156,7.4165308e+03,1.3224514e+01,1.9003071e+03,6.6757946e+00,8.0353765e+03,1.4611210e+01,0,nan,nan,nan,nan,743.37142,1.6582146e-03,1030.75156,1.5672606e-03,3.9467834e-02,8.1147596e-02
2458765.0 .. -0.3052998392563495,8.0455757e+03,1.4600376e+01,4.7049341e-03,386809,743.37579,1030.75775,7.4285151e+03,1.3214708e+01,1.8975846e+03,6.6496892e+00,8.0455757e+03,1.4600376e+01,0,nan,nan,nan,nan,743.37579,1.6511347e-03,1030.75775,1.5659113e-03,4.2884134e-02,8.6799361e-02
2458765.0 .. -0.3039109435198952,8.0578491e+03,1.4611950e+01,4.7049415e-03,386810,743.38068,1030.75563,7.4400063e+03,1.3225183e+01,1.8851906e+03,6.6451707e+00,8.0578491e+03,1.4611950e+01,0,nan,nan,nan,nan,743.38068,1.6487412e-03,1030.75563,1.5666722e-03,4.6368241e-02,8.7138519e-02
2458765.0 .. -0.30252204731777965,8.0199448e+03,1.4623798e+01,4.7049494e-03,386811,743.38077,1030.75486,7.4108506e+03,1.3235908e+01,1.8932153e+03,6.6664557e+00,8.0199448e+03,1.4623798e+01,0,nan,nan,nan,nan,743.38077,1.6638068e-03,1030.75486,1.5711738e-03,4.3958057e-02,8.7265849e-02
2458765.0 .. -0.3011331511154367,8.0214658e+03,1.4593859e+01,4.7049574e-03,386812,743.37518,1030.75434,7.4129116e+03,1.3208809e+01,1.8983730e+03,6.6483836e+00,8.0214658e+03,1.4593859e+01,0,nan,nan,nan,nan,743.37518,1.6544337e-03,1030.75434,1.5672805e-03,4.4078931e-02,8.4882207e-02


### Retrieve periodogram information

In [25]:
read_hdul['GLS_PG']

In [26]:
read_hdul['GLS_PG'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   16 / length of dimension 1                          
NAXIS2  =               285601 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    2 / number of table fields                         
TTYPE1  = 'freq    '                                                            
TFORM1  = 'D       '                                                            
TUNIT1  = 'd-1     '                                                            
TTYPE2  = 'power   '                                                            
TFORM2  = 'D       '        

In [27]:
read_hdul['GLS_PG'].data[:10]

FITS_rec([(2.4  , 0.00036205), (2.401, 0.00034928), (2.402, 0.00033384),
          (2.403, 0.00031599), (2.404, 0.00029605), (2.405, 0.00027439),
          (2.406, 0.0002514 ), (2.407, 0.00022753), (2.408, 0.00020322),
          (2.409, 0.00017894)],
         dtype=(numpy.record, [('freq', '>f8'), ('power', '>f8')]))

In [28]:
read_pg_table = QTable.read(read_hdul['GLS_PG'])
read_pg_table

freq,power
1 / d,
float64,float64
2.4,0.0003620468555181106
2.401,0.0003492843937411185
2.4019999999999997,0.0003338406612508559
2.403,0.0003159883997529411
2.404,0.0002960486655904126
2.405,0.00027438570758704726
2.4059999999999997,0.000251401013662915
2.407,0.00022752661318749132


In [29]:
read_pg_table['freq']

<Quantity [  2.4  ,   2.401,   2.402, ..., 287.998, 287.999, 288.   ] 1 / d>

In [30]:
diff_frecs = read_pg_table['freq'][:-1] - read_pg_table['freq'][1:]
diff_frecs

<Quantity [-0.001, -0.001, -0.001, ..., -0.001, -0.001, -0.001] 1 / d>

### Delete read information

In [31]:
read_hdul.close()

In [32]:
del read_hdul

## Summary of calculated periodograms

Number of objects with a TESS file available:

In [33]:
gto[~gto['lc_file'].isna()]['lc_file']

2      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2354...
3      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2350...
5      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2343...
7      ../data/CARMENES_GTO_TESS_lc/lightcurves/J2338...
10     ../data/CARMENES_GTO_TESS_lc/lightcurves/J2324...
                             ...                        
381    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0040...
382    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0038...
383    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0028...
385    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0018...
389    ../data/CARMENES_GTO_TESS_lc/lightcurves/J0005...
Name: lc_file, Length: 271, dtype: object

There are 271 objects in the initial sample that do have a TESS light curve file available.

Number of objects with TESS PG properly calculated:

In [34]:
gto[gto['valid_PG_TESS'] == 1.0]

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_T0_PG_TESS,WF_e_T0_PG_TESS,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.689208,0.027575,999.999756,2.151008e-06,1.0,1.0,NaN,94.758838,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23548+38...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1353.926624,0.036481,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
5,J23431+365,GJ 1289,-,1289,355.781509,36.536310,23:43:06.31,+36:32:13.1,107.922839,-24.336479,...,1764.702841,0.028355,999.999512,4.306074e-06,1.0,1.0,NaN,97.939914,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23431+36...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
7,J23381-162,G 273-093,-,4352,354.532687,-16.236514,23:38:08.16,-16:14:10.2,61.845437,-69.825220,...,1353.926631,0.036480,1000.000122,9.022946e-07,1.0,1.0,NaN,136.603404,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23381-16...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
10,J23245+578,BD+57 2735,-,895,351.126628,57.853057,23:24:30.51,+57:51:15.5,111.552287,-3.085183,...,1955.851155,0.018437,999.999512,3.720858e-06,1.0,1.0,NaN,131.327304,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23245+57...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,J00403+612,2MASS J00402129+6112490,-,NaN,10.089157,61.213379,00:40:21.29,+61:12:49.1,121.597733,-1.629661,...,1764.687712,NaN,1000.000000,NaN,NaN,1.0,NaN,176.113404,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00403+61...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
382,J00389+306,Wolf 1056,-,26,9.754064,30.616338,00:38:59.04,+30:36:58.4,119.774312,-32.179243,...,1764.682476,0.027265,999.999878,1.071631e-06,1.0,1.0,NaN,191.575017,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00389+30...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
383,J00286-066,GJ 1012,-,1012,7.162970,-6.667219,00:28:39.46,-06:39:49.2,107.085752,-68.835040,...,1385.744584,0.030548,1000.000305,2.678387e-06,1.0,1.0,NaN,175.054456,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00286-06...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...
385,J00183+440,GX And,A,15A,4.613226,44.024787,00:18:22.88,+44:01:22.6,116.690592,-18.446865,...,1764.644951,0.030860,999.999756,2.134945e-06,1.0,1.0,NaN,191.241441,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J00183+44...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...


But there are only 267 objects out of the 271 for wich periodogram was calculated without trouble. Let's see the exceptions:

In [35]:
len(gto[(gto['valid_PG_TESS'] == 1.0) & (~gto['lc_file'].isna())])

267

In [36]:
gto[(gto['valid_PG_TESS'] == 0.0) & (~gto['lc_file'].isna())][['Karmn', 'lc_file', 'error_PG_TESS']]

,Karmn,lc_file,error_PG_TESS
33,J22012+283,../data/CARMENES_GTO_TESS_lc/lightcurves/J2201...,Floating point nan values are not allowed in F...
49,J20450+444,../data/CARMENES_GTO_TESS_lc/lightcurves/J2045...,Floating point nan values are not allowed in F...
62,J19251+283,../data/CARMENES_GTO_TESS_lc/lightcurves/J1925...,Floating point nan values are not allowed in F...
362,J02070+496,../data/CARMENES_GTO_TESS_lc/lightcurves/J0207...,Floating point nan values are not allowed in F...


In [37]:
gto[(gto['valid_PG_TESS'] == 0.0) & (~gto['lc_file'].isna())]['lc_file'].values

array(['../data/CARMENES_GTO_TESS_lc/lightcurves/J22012+283/J22012+283_TESS_TIC283410775_S15.fits',
       '../data/CARMENES_GTO_TESS_lc/lightcurves/J20450+444/J20450+444_TESS_TIC239332587_S15.fits',
       '../data/CARMENES_GTO_TESS_lc/lightcurves/J19251+283/J19251+283_TESS_TIC110220637_S14.fits',
       '../data/CARMENES_GTO_TESS_lc/lightcurves/J02070+496/J02070+496_TESS_TIC250559611_S18.fits'],
      dtype=object)

In [38]:
gto[(gto['valid_PG_TESS'] == 0.0) & (~gto['lc_file'].isna())]['error_PG_TESS'].values

array(['Floating point nan values are not allowed in FITS headers.',
       'Floating point nan values are not allowed in FITS headers.',
       'Floating point nan values are not allowed in FITS headers.',
       ValueError('Floating point nan values are not allowed in FITS headers.')],
      dtype=object)

Maybe it seems to be something to do with the generation of the fits files.

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic GLS periodogram calculation for the available TESS light curves for GTO objetcs, and stored the results.
- Out of the initial 390 objects:
  - 271 stars do have a TESS light curve available. Of these:
    - 267 stars had their TESS periodogram calculated correctly.
    - 4 stars gave trouble during periodogram calculation. Probably it has something to do with the code (fits header generation), rather than the data in the star itself. Given the low impact, these 4 exceptions won't be further analyzed:
      - J22012+283
      - J20450+444
      - J19251+283
      - J02070+496